In [1]:
import numpy
import pandas
import numpy
from math import log, exp
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn.metrics import accuracy_score
import xgboost as xgb
import math

In [2]:
# Load Training Data
dataframe_training = pandas.read_csv("Data/train.csv",sep=",",delimiter=None, delim_whitespace=False, header=0)
print ("Original Model Shape: ",dataframe_training.shape)   
dataframe_training.head(3)

Original Model Shape:  (400, 15)


,id,penalty,l1_ratio,alpha,max_iter,random_state,n_jobs,n_samples,n_features,n_classes,n_clusters_per_class,n_informative,flip_y,scale,time
0,0,none,0.304083,0.0001,417,475,-1,1089,327,4,3,7,0.074798,24.242009,0.409987
1,1,l1,0.727744,0.0010,578,569,1,790,373,4,5,7,0.077781,54.626302,3.950953
2,2,none,0.745885,0.0100,588,529,2,428,1198,2,5,6,0.030196,17.999964,0.368702


In [3]:
# Get-dummies and do Feature Engineering part
dataframe_training = pandas.read_csv("Data/train.csv",sep=",",delimiter=None, delim_whitespace=False, header=0)
dataframe_training = pandas.get_dummies(dataframe_training)
print ("After pandas.get_dummies Model Shape: ",dataframe_training.shape)

def FeatureEngineering(input_dataframe):
    input_dataframe['n_jobs']=input_dataframe['n_jobs'].mask(input_dataframe['n_jobs'] == -1, 8)
    input_dataframe["mix_feature1"]=input_dataframe['n_samples']*input_dataframe['n_classes']*input_dataframe['n_features']/input_dataframe['n_jobs']
    input_dataframe["mix_feature2"]=input_dataframe['n_samples']*input_dataframe['n_classes']*input_dataframe['max_iter']/input_dataframe['n_jobs']
    input_dataframe["mix_feature3"]=input_dataframe['n_samples']*input_dataframe['n_features']*input_dataframe['max_iter']
    input_dataframe["mix_feature4"]=input_dataframe['n_samples']*input_dataframe['n_classes']*input_dataframe['n_features']*input_dataframe['max_iter']/input_dataframe['n_jobs']
    input_dataframe["penalty_l2"]=(1-input_dataframe['l1_ratio'])*input_dataframe['penalty_l2']
    input_dataframe["penalty_none"]=(1-input_dataframe['l1_ratio'])*input_dataframe['penalty_none']
    input_dataframe["penalty_elasticnet"]=input_dataframe['l1_ratio']*input_dataframe['penalty_elasticnet']
    return input_dataframe

dataframe_training = FeatureEngineering(dataframe_training)


FeatureToDrop= ['random_state','scale','n_features','n_classes','n_samples','n_jobs','max_iter','alpha','n_clusters_per_class','l1_ratio']


dataframe_training=dataframe_training.drop(FeatureToDrop, axis=1)
dataframe_training.head(3)

After pandas.get_dummies Model Shape:  (400, 18)


,id,n_informative,flip_y,time,penalty_elasticnet,penalty_l1,penalty_l2,penalty_none,mix_feature1,mix_feature2,mix_feature3,mix_feature4
0,0,7,0.074798,0.409987,0.0,0,0.0,0.695917,178051.5,227056.5,148494951,74247475.5
1,1,7,0.077781,3.950953,0.0,1,0.0,0.000000,1178680.0,1826480.0,170319260,681277040.0
2,2,6,0.030196,0.368702,0.0,0,0.0,0.254115,512744.0,251664.0,301493472,301493472.0


In [6]:
X = dataframe_training.drop(['time'], axis=1).values 
Y = dataframe_training.filter(items=['time']).values
# Split Data into Train and Test 
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=7)
print (X_train.shape,len(Y_train),len(X_test),Y_test.shape)

(300, 11) 300 100 (100, 1)


In [7]:
# Use Linear Regressioon Model to Predict
from sklearn.ensemble import *
from sklearn.linear_model import *
ToRunModels={LinearRegression}

In [8]:
# Drop time from Testing Data
X = dataframe_training.drop(['time'], axis=1).values 
Y = dataframe_training.filter(items=['time']).values 

#Try to take Log so as to have a better result
for temp_index,temp_item in enumerate(Y):
    Y[temp_index]=log(Y[temp_index])

In [9]:
# Load Test Dataset
dataframe_test = pandas.read_csv("Data/test.csv",sep=",",delimiter=None, delim_whitespace=False, header=0)
print ("Original Model Shape: ",dataframe_test.shape)   

dataframe_test = pandas.get_dummies(dataframe_test)
print ("After pandas.get_dummies Model Shape: ",dataframe_test.shape) 

dataframe_test = FeatureEngineering(dataframe_test)
dataframe_test=dataframe_test.drop(FeatureToDrop, axis=1)  

X_RealTest = dataframe_test.values 
dataframe_test.head(5)

Original Model Shape:  (100, 14)
After pandas.get_dummies Model Shape:  (100, 17)


,id,n_informative,flip_y,penalty_elasticnet,penalty_l1,penalty_l2,penalty_none,mix_feature1,mix_feature2,mix_feature3,mix_feature4
0,0,8,0.049072,0.000000,0,0.309616,0.000000,1.112496e+06,621264.0000,892963456,6.697226e+08
1,1,11,0.041393,0.215088,0,0.000000,0.000000,2.284360e+06,780390.0000,476974368,1.192436e+09
2,2,8,0.005987,0.000000,0,0.782542,0.000000,2.787120e+05,110047.0000,221854752,1.109274e+08
3,3,10,0.002964,0.000000,0,0.000000,0.879697,4.889368e+05,162731.8125,1144764057,2.146433e+08
4,4,8,0.050126,0.000000,0,0.000000,0.541149,4.889115e+06,888349.0000,776670840,1.359174e+09


In [11]:
df_test=  dataframe_training[(dataframe_training.penalty_elasticnet != 0)].drop(['penalty_l1','penalty_l2','penalty_none','id'], axis=1)
df_test=  dataframe_training[(dataframe_training.penalty_l1 != 0)].drop(['penalty_elasticnet','penalty_l2','penalty_none','id'], axis=1)
df_test=  dataframe_training[(dataframe_training.penalty_l2 != 0)].drop(['penalty_l1','penalty_elasticnet','penalty_none','id'], axis=1)
df_test=  dataframe_training[(dataframe_training.penalty_none != 0)].drop(['penalty_l1','penalty_l2','penalty_elasticnet','id'], axis=1)

In [12]:
#Training Data Extration
X_Tr_EN = dataframe_training[(dataframe_training.penalty_elasticnet != 0)].drop(['time','penalty_l1','penalty_l2','penalty_none','id'], axis=1).values 
Y_Tr_EN = dataframe_training[(dataframe_training.penalty_elasticnet != 0)].filter(items=['time']).values 
 
X_Tr_L1 = dataframe_training[(dataframe_training.penalty_l1 != 0)].drop(['time','penalty_elasticnet','penalty_l2','penalty_none','id'], axis=1).values 
Y_Tr_L1 = dataframe_training[(dataframe_training.penalty_l1 != 0)].filter(items=['time']).values 

X_Tr_L2 = dataframe_training[(dataframe_training.penalty_l2 != 0)].drop(['time','penalty_elasticnet','penalty_l1','penalty_none','id'], axis=1).values 
Y_Tr_L2 = dataframe_training[(dataframe_training.penalty_l2 != 0)].filter(items=['time']).values 

X_Tr_NA = dataframe_training[(dataframe_training.penalty_none != 0)].drop(['time','penalty_elasticnet','penalty_l1','penalty_l2','id'], axis=1).values 
Y_Tr_NA = dataframe_training[(dataframe_training.penalty_none != 0)].filter(items=['time']).values 

In [13]:
#Test Data Extration
X_Test_EN = dataframe_test[(dataframe_test.penalty_elasticnet != 0)].drop(['penalty_l1','penalty_l2','penalty_none','id'], axis=1).values 
X_Test_EN_id = dataframe_test[(dataframe_test.penalty_elasticnet != 0)].filter(items=['id']).values 

X_Test_L1 = dataframe_test[(dataframe_test.penalty_l1 != 0)].drop(['penalty_elasticnet','penalty_l2','penalty_none','id'], axis=1).values 
X_Test_L1_id = dataframe_test[(dataframe_test.penalty_l1 != 0)].filter(items=['id']).values 

X_Test_L2 = dataframe_test[(dataframe_test.penalty_l2 != 0)].drop(['penalty_elasticnet','penalty_l1','penalty_none','id'], axis=1).values 
X_Test_L2_id = dataframe_test[(dataframe_test.penalty_l2 != 0)].filter(items=['id']).values 

X_Test_NA = dataframe_test[(dataframe_test.penalty_none != 0)].drop(['penalty_elasticnet','penalty_l1','penalty_l2','id'], axis=1).values 
X_Test_NA_id = dataframe_test[(dataframe_test.penalty_none != 0)].filter(items=['id']).values 

#Initialize 
Y_Predicted =[]
Y_Predicted_id=[]

Y_Predicted_id.extend(X_Test_EN_id)
Y_Predicted_id.extend(X_Test_L1_id)
Y_Predicted_id.extend(X_Test_L2_id)
Y_Predicted_id.extend(X_Test_NA_id)

In [14]:
ModelToRunTime = 1
Accumulated_Y_Predicted =[]
Y_Predicted =[]
ToRunModels=LinearRegression
RunTimeCounter=0
for tempj in range(0, ModelToRunTime):
  #Total Result In this Round
  temp_Y_Predicted = []
  RunTimeCounter+=1
  
  #From Elasticnet Data 
  tempmodel=ToRunModels()
  tempmodel.fit(X_Tr_EN, Y_Tr_EN.ravel())
  tempActualLOGYPredicted =tempmodel.predict(X_Test_EN)
  tempActualYPredicted=tempActualLOGYPredicted
  temp_Y_Predicted.extend(tempActualYPredicted)
  
  #From L1 Data 
  tempmodel=ToRunModels()
  tempmodel.fit(X_Tr_L1, Y_Tr_L1.ravel())
  tempActualLOGYPredicted =tempmodel.predict(X_Test_L1)
  tempActualYPredicted=tempActualLOGYPredicted
  temp_Y_Predicted.extend(tempActualYPredicted)

  #From L2 Data 
  tempmodel=ToRunModels()
  tempmodel.fit(X_Tr_L2, Y_Tr_L2.ravel())
  tempActualLOGYPredicted =tempmodel.predict(X_Test_L2)
  tempActualYPredicted=tempActualLOGYPredicted
  temp_Y_Predicted.extend(tempActualYPredicted)

  #From None Data 
  tempmodel=ToRunModels()
  tempmodel.fit(X_Tr_NA, Y_Tr_NA.ravel())
  tempActualLOGYPredicted =tempmodel.predict(X_Test_NA)
  tempActualYPredicted=tempActualLOGYPredicted
  temp_Y_Predicted.extend(tempActualYPredicted)
    
  #Store Everything at Accumulated_Y_Predicted
  if len(Accumulated_Y_Predicted)==0:
    Accumulated_Y_Predicted.extend(temp_Y_Predicted)
    Y_Predicted.extend(temp_Y_Predicted)
  else:
    print("Accumulating")
    for temp_index,temp_item in enumerate(Accumulated_Y_Predicted):
      Accumulated_Y_Predicted[temp_index]+=temp_Y_Predicted[temp_index]

for temp_index,temp_item in enumerate(Accumulated_Y_Predicted):
  Y_Predicted[temp_index]=Accumulated_Y_Predicted[temp_index]/ModelToRunTime
  
print ("RunTimeCounter:",RunTimeCounter)
print (Y_Predicted)

RunTimeCounter: 1
[8.746392522486897, 12.086663848870392, 14.567750672275968, 34.00525871597447, 0.21831650707934663, 0.6217648574452009, 3.67768382466765, 1.1760806369818415, 2.1771434304921256, 18.86993865064229, 7.202197354444599, 2.5943913683739463, 1.0475297416112015, 1.8834374194796215, 5.601652494170726, 0.42620575159682583, 1.0914880378413163, 5.9238712405008265, 8.804858213601966, 3.0105306975586683, 1.3867207218186408, 10.237371676043962, 3.3856177134968233, 8.84989179855636, 6.4322763117444515, 2.85885478956637, 7.449407224419994, 1.4467268977457248, 0.436954302484162, 3.0163643302186545, 2.150146463172736, 7.570711504725245, 0.9131113790718652, 2.0175797163777958, 3.860762353627875, 8.247062357604342, 3.47125149485903, 17.613654573690493, 0.22145327200108345, 8.129528943122054, 10.941413643716135, 5.485955701020872, 0.7641272248675939, 1.5685018282083991, 4.987381952416904, 12.706229502323374, 47.67958539219502, 1.2999351508061903, 0.2259995597319372, 5.141123731700974, 0.6

In [15]:
print("Id,time")
for tempi in range(0, 100):
  for tempj in range(0, 100):
    if tempi == (Y_Predicted_id[tempj]):
      print ("%s,%s"%(Y_Predicted_id[tempj][0],Y_Predicted[tempj]))

Id,time
0,1.2999351508061903
1,8.746392522486897
2,0.2259995597319372
3,1.0356602637701497
4,2.0871021473177307
5,7.449407224419994
6,2.35498521267652
7,0.4408938781359108
8,12.086663848870392
9,0.4621917168750029
10,5.141123731700974
11,14.567750672275968
12,0.636345806550624
13,34.00525871597447
14,0.21831650707934663
15,0.8171444994939496
16,0.6217648574452009
17,6.049734354564874
18,3.67768382466765
19,1.4467268977457248
20,0.30629452219575204
21,0.3235657571712931
22,0.5370889937950069
23,0.436954302484162
24,1.44378365456045
25,1.3210343906056885
26,1.1760806369818415
27,3.0163643302186545
28,2.1771434304921256
29,2.7875572484705344
30,18.86993865064229
31,2.150146463172736
32,0.3822025256246788
33,7.570711504725245
34,7.202197354444599
35,0.9131113790718652
36,2.5943913683739463
37,2.0175797163777958
38,0.7644916479666795
39,2.253975989352106
40,3.860762353627875
41,8.247062357604342
42,1.0475297416112015
43,3.1706298993845063
44,0.7557500669126211
45,3.47125149485903
46,1.88343